In [ ]:
# TODO: Do EDA of NUM_SAMPLES of the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# WIP: perform the true/false correlation analysis for these score columns: MSGFScore, IsotopeError, PrecursorError, DeNovoScore, SpecEValue, EValue
def analyze_csv(file_path, score_column='score', similarity_threshold=0.1):
    # Load CSV data
    df = pd.read_csv(file_path)
    
    # Separate true and false rows
    true_rows = df.iloc[::5]
    false_rows = df.drop(true_rows.index)

    true_scores = true_rows[score_column]
    false_scores = false_rows[score_column]

    # Calculate summary statistics
    true_mean = true_scores.mean()
    true_std = true_scores.std()
    false_mean = false_scores.mean()
    false_std = false_scores.std()

    print(f"True rows mean: {true_mean}, std: {true_std}")
    print(f"False rows mean: {false_mean}, std: {false_std}")

    # Compare false row scores with their associated true row
    false_rows['true_score'] = np.repeat(true_scores.values, 4)
    false_rows['score_diff'] = np.abs(false_rows['true_score'] - false_rows[score_column])

    close_rows = false_rows[false_rows['score_diff'] <= similarity_threshold]
    print("\nFalse rows with similar scores to their associated true rows:")
    print(close_rows)

    # 1. Score distribution
    plt.figure()
    plt.hist(true_scores, bins=30, alpha=0.5, label='True rows', color='green')
    plt.hist(false_scores, bins=30, alpha=0.5, label='False rows', color='red')
    plt.xlabel('Score')
    plt.ylabel('Frequency')
    plt.legend()
    plt.title('Score Distribution')
    plt.show()

    # 8. Length analysis
    df['peptide_length'] = df['peptide'].apply(len)
    length_vs_score = df.groupby('peptide_length')[score_column].mean()
    plt.figure()
    plt.plot(length_vs_score, marker='o', linestyle='--')
    plt.xlabel('Peptide Length')
    plt.ylabel('Average Score')
    plt.title('Peptide Length vs Average Score')
    plt.show()

    # 9. Modification analysis
    modification_columns = ['mod_C', 'mod_M', 'mod_N']  # Replace with actual modification columns
    modification_scores = df.groupby(modification_columns)[score_column].mean()
    print("\nAverage scores by modifications:")
    print(modification_scores)

    # True vs False Row Scores
    plt.figure()
    plt.scatter(true_rows.index, true_scores, label='True rows', color='green', marker='o')
    plt.scatter(false_rows.index, false_scores, label='False rows', color='red', marker='x')
    plt.xlabel('Row index')
    plt.ylabel('Score')
    plt.legend()
    plt.title('True vs False Row Scores')
    plt.show()

# Example usage
analyze_csv('path/to/your/csvfile.csv', score_column='score', similarity_threshold=0.1)
